In [2]:
from brfinance import CVMAsyncBackend
import pandas
from datetime import datetime, date
import requests
from bs4 import BeautifulSoup

In [3]:
class CVM():

    def __init__(self) -> None:

        self.cvm_http_client = CVMAsyncBackend()

    def firms_informations(self, extract_data:bool = True, fetch_data:bool = True, fetch_br_fin:bool = False, record_file:str = "EmpresasBrasilCVM"):

        """ ADQUIRE INFORMAÇÕES DE COMPANHIAS ABERTAS BRASILEIRAS VIA CVM

            REFERENCIA DE SITE: https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CiaAb/FormBuscaCiaAbOrdAlf.aspx?LetraInicial=3

            EXTRACT_DATA [BOOL] > EXTRAI OS DADOS EM TABELAS, SEM CABEÇALHO E CRIA ARQUIVO DE REFERENCIA DE DADOS EM FORMATO .TXT
            FETCH_DATA   [BOOL] > AJUSTA DATAFRAME DE REFERÊNCIA CVM
            FETCH_BR_FIN [BOOL] > VERIFICA AS INFORMAÇÕES PRESENTES NA LIB BR_FINANCE
        """
        if extract_data:
            # INFORMAÇÕES SOBRE AS COMPANHIAS BRASILEIRAS
            infos = []

            # TODAS AS LETRAS MAIUSCULAS E NUMEROS
            # CONFORME MOSTRADO NO BUSCADOR DA CVM
            capital_letters_and_numbers = []

            # TODAS AS LETRAS MAIUSCULAS VIA ASCII
            for ascii_value in range(ord('A'), ord('Z') + 1):
                capital_letters_and_numbers.append(chr(ascii_value))

            # TODOS OS DIGITOS POSSIVEIS
            for digit in range(10):
                capital_letters_and_numbers.append(str(digit))

            # PARA CADA POSSIBILIDADE NAS LISTAS CRIADAS
            # TENTAMOS ACESSAR AS TABELAS DISPONIVEIS
            for focus in capital_letters_and_numbers:

                # USA O SITE BASE DA CVM E ALTERA A LETRA INICIAL PARA BUSCAR AS TABELAS
                url = f'https://cvmweb.cvm.gov.br//SWB//Sistemas//SCW//CPublica//CiaAb//FormBuscaCiaAbOrdAlf.aspx?LetraInicial={focus}'
                print(f"Researching firms: Start with {focus}")

                # USA O REQUEST PARA ACESSAR A URL
                response = requests.get(url)

                # SE O STATUS CODE = 200, ACESSO FOI UM SUCESSO
                if response.status_code == 200:

                    try:

                        # REALIZA O PARSING DO HTML VIA BEAUTIFULSOUP
                        html_content = response.text
                        soup = BeautifulSoup(html_content, 'html.parser')

                        # TODAS AS TABELAS NO HTML
                        tables = soup.find_all('table')

                        for table in tables:

                            # PROCESSAMOS OS ELEMENTOS 'TABELA'
                            rows = table.find_all('tr')

                            for row in rows:

                                # CÉLULAS DA TABELA
                                cells = row.find_all(['th', 'td'])
                                # TEXTO COMPLETO DE INFORMAÇÕES A SER EXTRAÍDO
                                full_txt = ""

                                for cell in cells:

                                    # ADQUIRE O TEXTO, SEM ESPAÇOS
                                    txt = cell.get_text(strip = True)

                                    if not txt.startswith(("CNPJ", "Selecione", " ")):

                                        # NÃO DESEJAMOS OS CABEÇALHOS PADRÃO
                                        full_txt += txt + ";"

                                    else: break

                                # CASO O TEXTO SEJA VALIDO, ACRESCENTA O ARQUIVO EMPRESAS BRASIL
                                if len(full_txt) > 0:

                                    infos.append(full_txt)                     

                    except Exception as exc:

                        # NOS AJUDA A COMPREENDER SE HOUVE UM ERRO E PORQUE
                        print(f"[ERROR] at {focus}: {exc}")

                else:

                    # NÃO FOI POSSIVEL REALIZAR O REQUEST
                    print(f"[WARNING] at {focus}: Response Status - {response.status_code}")

            # CRIA O ARQUIVO FINAL DE INFORMAÇÕES
            with open(record_file + ".txt", "w+") as record:
                for info in infos:
                    try:
                        record.write(info + "\n")
                    except Exception as exc:
                        print(info, exc)

        if fetch_data:

            # MESMA REFERÊNCIA DE FORMATO UTILIZADA PELO BrFINANCE
            def format_number(number):
                return f'{number:06d}'

            # EXTRAI O DATAFRAME DE INFORMAÇÕES DA CVM
            df = pandas.read_csv(
                record_file + ".txt", sep = ";", encoding = "latin1", header = None
            ).dropna(axis = 1)
            # NOME PADRÃO DAS COLUNAS
            df.columns = ["CNPJ", "FIRM_NAME", "CVM_FIRM_TYPE", "CVM_CODE", "CVM_STATUS"]
            # CORRIGE OS CÓDIGOS
            df["CVM_CODE"] = df["CVM_CODE"].apply(format_number)
            # SOBSCREVE AS INFORMAÇÕES
            df.to_csv(record_file + ".txt", index = False)

        if fetch_br_fin:

            cvm_codes = self.cvm_http_client.get_cvm_codes()
            df = pandas.read_csv(record_file + ".txt")
            print(cvm_codes)
            print(df)

In [4]:
cvm = CVM()

In [5]:
cvm.firms_informations(extract_data = False, fetch_data = False, fetch_br_fin = True)

ConnectionError: HTTPSConnectionPool(host='www.rad.cvm.gov.br', port=443): Max retries exceeded with url: /ENET/frmConsultaExternaCVM.aspx (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F17F6F6B90>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [6]:
cvm_http_client = CVMAsyncBackend()

In [10]:
for i in range(3):
    try:
        codes = cvm_http_client.get_cvm_codes()
        break
    except Exception as exc: print(exc)
codes

HTTPSConnectionPool(host='www.rad.cvm.gov.br', port=443): Max retries exceeded with url: /ENET/frmConsultaExternaCVM.aspx (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F10E0830D0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))
HTTPSConnectionPool(host='www.rad.cvm.gov.br', port=443): Max retries exceeded with url: /ENET/frmConsultaExternaCVM.aspx (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F10E083EE0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))
HTTPSConnectionPool(host='www.rad.cvm.gov.br', port=443): Max retries exceeded with url: /ENET/frmConsultaExternaCVM.aspx (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F10E0837C0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))


NameError: name 'codes' is not defined

In [35]:


df = pandas.DataFrame.from_dict(cvm_codes, orient = "index")
df.columns = ["CompanyName"]
df["CVM"] = df.index
df

,CompanyName,CVM
018554,"CRÉDITO, FINANCIAMENTO E INVESTIMENTO (REGISTR...",018554
900049,1461 INVESTIMENTOS S.A (REGISTRO ATIVO),900049
252224,2W ECOBANK S.A.,252224
025224,2W ECOBANK S.A. (REGISTRO ATIVO),025224
917581,2W ENERGIA S.A. (REGISTRO ATIVO),917581
...,...,...
056456,ZOOM VIDEO COMMUNICATIONS INC (REGISTRO ATIVO),056456
058955,ZOOMINFO TECHNOLOGIES INC (REGISTRO ATIVO),058955
058963,ZSCALER INC (REGISTRO ATIVO),058963
057169,ZTO EXPRESS CAYMAN INC (REGISTRO ATIVO),057169


In [41]:
cvm_codes['025291']

'3R PETROLEUM ÓLEO E GÁS S.A (REGISTRO ATIVO)'

In [9]:
categories = cvm_http_client.get_consulta_externa_cvm_categories()
#pandas.DataFrame.from_dict(categories, orient = "index").to_excel("Categories.xlsx")
categories

ConnectionError: HTTPSConnectionPool(host='www.rad.cvm.gov.br', port=443): Max retries exceeded with url: /ENET/frmConsultaExternaCVM.aspx (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F10E083DF0>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [6]:
# Realizando busca por Empresa
start_date = date(2000, 1, 1)
end_dt = date.today()
cvm_codes_list = ['7617'] # B3 '21610'
#category = ["EST_4", "EST_3", "IPE_4_-1_-1"] # Códigos de categoria para DFP, ITR e fatos relevantes
category = list(categories.keys())
last_ref_date = False # Se "True" retorna apenas o último report no intervalo de datas

In [7]:
# Busca
search_result = cvm_http_client.get_consulta_externa_cvm_results(
    cod_cvm = cvm_codes_list,
    start_date = start_date,
    end_date = end_dt,
    last_ref_date = last_ref_date,
    category = categories
)

c:\Users\kevin.pergher\AppData\Local\anaconda3\envs\Quantitas310\lib\site-packages\brfinance\responses.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  response_df = response_df.append(search_results_df)


In [9]:
search_result.head()

,cod_cvm,empresa,categoria,tipo,especie,ref_date,data_entrega,status,version,modalidade,acoes,outros,view_url,numero_seq_documento,codigo_tipo_instituicao,numSequencia,numVersao,numProtocolo,descTipo
2469,007617,ITAÚSA S.A.,FRE - Formulário de Referência,-,-,2023-12-31,2023-10-10 18:43:00,Ativo,2,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,131375,1,131375,2,007617FRE202320230200131375-65,
2468,007617,ITAUSA S.A.,Comunicado ao Mercado,Apresentações a analistas/agentes do mercado,Panorama Itaúsa 2023 -,2023-10-10,2023-10-10 08:57:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmExibirArqui...,NaN,NaN,674827,1,1150101,IPE
2467,007617,ITAUSA S.A.,Valores Mobiliários Negociados e Detidos,"Posição Individual - Cia, Controladas e Coligadas",-,2023-09-01,2023-10-09 18:17:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmExibirArqui...,NaN,NaN,674649,1,1149923,IPE
2466,007617,ITAUSA S.A.,Valores Mobiliários Negociados e Detidos,Posição Consolidada,-,2023-09-01,2023-10-09 18:15:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmExibirArqui...,NaN,NaN,674645,1,1149919,IPE
2465,007617,ITAUSA S.A.,Reunião da Administração,Conselho Fiscal,Alteração da composição do Conselho FiscalAta,2023-10-02,2023-10-03 18:07:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmExibirArqui...,NaN,NaN,673071,1,1148345,IPE


In [22]:
search_result.iloc[0].view_url

'https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?NumeroSequencialDocumento=131375&CodigoTipoInstituicao=1'

In [16]:
search_result.columns

Index(['cod_cvm', 'empresa', 'categoria', 'tipo', 'especie', 'ref_date',
       'data_entrega', 'status', 'version', 'modalidade', 'acoes', 'outros',
       'view_url', 'numero_seq_documento', 'codigo_tipo_instituicao',
       'numSequencia', 'numVersao', 'numProtocolo', 'descTipo'],
      dtype='object')

In [19]:
search_result.categoria.unique()

array(['FRE - Formulário de Referência', 'Comunicado ao Mercado',
       'Valores Mobiliários Negociados e Detidos',
       'Reunião da Administração', 'Aviso aos Acionistas',
       'Aviso aos Debenturistas', 'Fato Relevante', 'Relatório Proventos',
       'Calendário de Eventos Corporativos',
       'Dados Econômico-Financeiros', 'ITR - Informações Trimestrais',
       'Comunicação sobre Transação entre Partes Relacionadas',
       'Informe do Código de Governança', 'FCA - Formulário Cadastral',
       'Regimento interno do Conselho Fiscal',
       'Regimento interno de Comitês',
       'Política de Transações entre Partes Relacionadas',
       'Regimento Interno do Comitê de Auditoria Estatutário',
       'Assembleia', 'Relato Integrado',
       'Plano de Remuneração Baseado em Ações (exceto Plano de Opções)',
       'Estatuto Social', 'DFP - Demonstrações Financeiras Padronizadas',
       'Política de indicação',
       'Política de Negociação das Ações da Companhia',
       'Polít

In [17]:
reports_list = [
    'Balanço Patrimonial Ativo',
    'Balanço Patrimonial Passivo',
    'Demonstração do Resultado',
    'Demonstração do Resultado Abrangente',
    'Demonstração do Fluxo de Caixa',
    'Demonstração das Mutações do Patrimônio Líquido',
    'Demonstração de Valor Adicionado'] # Se None retorna todos os demonstrativos disponíveis.

In [24]:
# Obter demonstrativos
for index, row in search_result.iterrows():
    try:
        #empresa = f"{row['cod_cvm']} - {cvm_codes[row['cod_cvm']]}"
        #print(empresa)
        if not pandas.isna(row["numero_seq_documento"]) and not pandas.isna(row["codigo_tipo_instituicao"]):
            print(row["categoria"])
            #reports = cvm_http_client.get_report(row["numero_seq_documento"], row["codigo_tipo_instituicao"], reports_list=None)
            #print(reports)
    except Exception as exc:
        print(exc)
    
    #for report in reports:
    #    reports[report]["cod_cvm"] = row["cod_cvm"]
    #    print(reports[report].head())

FRE - Formulário de Referência
ITR - Informações Trimestrais
FRE - Formulário de Referência
FCA - Formulário Cadastral
FRE - Formulário de Referência
ITR - Informações Trimestrais
FRE - Formulário de Referência
DFP - Demonstrações Financeiras Padronizadas
FRE - Formulário de Referência
FCA - Formulário Cadastral
FRE - Formulário de Referência
ITR - Informações Trimestrais
FRE - Formulário de Referência
FRE - Formulário de Referência
ITR - Informações Trimestrais
FRE - Formulário de Referência
ITR - Informações Trimestrais
ITR - Informações Trimestrais
FRE - Formulário de Referência
FRE - Formulário de Referência
DFP - Demonstrações Financeiras Padronizadas
FCA - Formulário Cadastral
FRE - Formulário de Referência
FRE - Formulário de Referência
ITR - Informações Trimestrais
FRE - Formulário de Referência
ITR - Informações Trimestrais
FRE - Formulário de Referência
FRE - Formulário de Referência
FCA - Formulário Cadastral
FRE - Formulário de Referência
FRE - Formulário de Referência
ITR 

In [15]:
search_result

,cod_cvm,empresa,categoria,tipo,especie,ref_date,data_entrega,status,version,modalidade,acoes,outros,view_url,numero_seq_documento,codigo_tipo_instituicao,numSequencia,numVersao,numProtocolo,descTipo
598,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2023-06-30,2023-08-14 20:24:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,130411,1,130411,1,007617ITR300620230100130411-79,ITR
603,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2023-03-31,2023-05-15 20:54:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,126807,1,126807,1,007617ITR310320230100126807-77,ITR
605,007617,ITAÚSA S.A.,DFP - Demonstrações Financeiras Padronizadas,-,-,2022-12-31,2023-03-20 19:47:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,124374,1,124374,1,007617DFP311220220100124374-60,DFP
609,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2022-09-30,2022-11-11 08:14:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,121843,1,121843,1,007617ITR300920220100121843-73,ITR
612,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2022-06-30,2022-08-15 19:39:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,119985,1,119985,1,007617ITR300620220100119985-71,ITR
615,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2022-03-31,2022-05-23 17:41:00,Ativo,2,RE,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,115662,1,115662,2,007617ITR310320220200115662-62,ITR
616,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2022-03-31,2022-05-16 18:13:00,Inativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,115312,1,115312,1,007617ITR310320220100115312-76,ITR
619,007617,ITAÚSA S.A.,DFP - Demonstrações Financeiras Padronizadas,-,-,2021-12-31,2022-02-14 19:05:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,111753,1,111753,1,007617DFP311220210100111753-73,DFP
623,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2021-09-30,2021-11-08 19:11:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,109518,1,109518,1,007617ITR300920210100109518-83,ITR
625,007617,ITAÚSA S.A.,ITR - Informações Trimestrais,-,-,2021-06-30,2021-08-09 18:49:00,Ativo,1,AP,<i class='fi-page-search' id='VisualizarDocume...,NaN,https://www.rad.cvm.gov.br/ENET/frmGerenciaPag...,107389,1,107389,1,007617ITR300620210100107389-78,ITR
